# Sandbox para seguir en vivo las clases

In [4]:
import numpy as np
import plotly.graph_objects as go

import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, fixed

## 2021-05-05

In [32]:
def is_positive_definite(matrix):
    if np.array_equal(matrix, matrix.T):
        try:
            np.linalg.cholesky(matrix)
            return True
        except np.linalg.LinAlgError:
            return False
    else:
        return False


def my_fun(x):
    return x[0] ** 2 + x[1] ** 2 - 2*x[0]*x[1]


def finite_difference(callable_fun, x_zero, direction, diff_size=0.01):
    d = np.zeros(len(x_zero))
    d[direction] = 1.0
    forward = callable_fun(x_zero + diff_size * d)
    backward = callable_fun(x_zero - diff_size * d)
    return (forward - backward) / (2 * diff_size)


def differential(callable_fun, x_zero, diff_size=0.01):
    rv = []
    for direction in range(len(x_zero)):
        rv.append(
            finite_difference(callable_fun, x_zero, direction, diff_size=diff_size)
        )
    return np.array(rv)


def hessian(callable_fun, x_zero, diff_size=0.01):
    rv = []
    for direction in range(len(x_zero)):
        partial = lambda x : finite_difference(my_fun, x, direction, diff_size=diff_size)
        rv.append(differential(partial, x_zero, diff_size=diff_size))
    return np.array(rv)

In [36]:
-hessian(my_fun, np.array([1, 1]))

array([[-2.,  2.],
       [ 2., -2.]])

## 2021-04-14

In [8]:
def sir_sim(infected_zero=0.001, g_zero=3, total_time=50, alpha=20):
    infected_s = [infected_zero]
    removed_s = [infected_zero]
    infected_h = infected_s.copy()
    removed_h = removed_s.copy()
    
    transmition_rate_s = g_zero * (1 + alpha * infected_s[0]) ** (-1)
    transmition_rate_h = g_zero * (1 + alpha * removed_h[0]) ** (-1)
    
    for week in range(1, total_time):
        new_infections_s = transmition_rate_s * infected_s[week-1] * (1 - removed_s[week-1])
        new_infections_h = transmition_rate_h * infected_h[week-1] * (1 - removed_h[week-1])
        
        infected_s.append(new_infections_s)
        infected_h.append(new_infections_h)
        
        removed_s.append(removed_s[week-1] + new_infections_s)
        removed_h.append(removed_h[week-1] + new_infections_h)
        
        transmition_rate_s = g_zero / (1 + alpha * infected_s[week])
        transmition_rate_h = g_zero / (1 + alpha * removed_h[week])

    time_axis = np.arange(0, total_time)
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=time_axis, y=infected_s, mode='lines', name='infected s.s.')
    )
    fig.add_trace(
        go.Scatter(x=time_axis, y=infected_h, mode='lines', name='removed h.a.')
    )
    fig.show()

In [11]:
sir_model = interactive(
    sir_sim,
    infected_zero=widgets.FloatSlider(value=0.01, min=0, max=1, step=0.001),
    g_zero=widgets.IntSlider(value=3, min=1, max=10, step=1),
    alpha=widgets.IntSlider(value=10, min=0, max=30, step=1),
    tota_time=widgets.IntSlider(value=36, min=24, max=96)
    
);
display(sir_model)

interactive(children=(FloatSlider(value=0.01, description='infected_zero', max=1.0, step=0.001), IntSlider(val…

## 2021-04-09

In [ ]:
# Diferencias centradas vs forward
def f(x):
    return abs(x-1)

def f2(x):
    return x ** 2

def f3(x):
    return x ** 3

h = np.logspace(-20, 0, 10000)  # numpy


x0 = 1
Dh = (f(x0+h) - f(x0)) / h
Dh2 = f(x0+h) - f(x0-h) / 2*h
plt.loglog(x0, Dh)